This notebook performs evaluation of the `tamil_mfd.dic` through the following steps:
1. Select 1 word from each MFD category (10 categories × 1 words = 10 total) from the Tamil MFD and obtain their corresponding English MFD terms.
2. Back-translate Tamil to English using Sarvam AI and compare
3. Human translate English to Tamil, compare with Machine Translated Tamil terms and calculate IAA

In [15]:
!git clone https://github.com/anagha1999/anlp-project/

fatal: destination path 'anlp-project' already exists and is not an empty directory.


In [16]:
import os

# Navigate to the repository directory
repo_dir = 'anlp-project'
if os.path.exists(repo_dir):
    os.chdir(repo_dir)
    print(f"Changed directory to: {os.getcwd()}")
    # Pull the latest changes from the remote repository
    !git pull
    # Navigate back to the original directory if needed
    os.chdir('..')
    print(f"Changed directory back to: {os.getcwd()}")
else:
    print(f"Repository directory '{repo_dir}' not found. Please ensure it is cloned.")

Changed directory to: /content/anlp-project
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 16 (delta 8), reused 16 (delta 8), pack-reused 0 (from 0)
Unpacking objects: 100% (16/16), 7.54 MiB | 7.15 MiB/s, done.
From https://github.com/anagha1999/anlp-project
   1458d71..c2463a1  main       -> origin/main
Updating 1458d71..c2463a1
Fast-forward
 kannada/2.Clean_Preprocess_Kannada_Dataset.ipynb   | 1000 +--
 kannada/3.Moral_Foundations_Kannada.ipynb          |  214 +-
 kannada/5.Eval_Moral_Extraction.ipynb              |    0
 kannada/kannada-dataset/3-siri-kannada.pdf         |  Bin 0 -> 7581798 bytes
 .../1-janapada-kathegalu.txt                       |    0
 .../2-niti-kathegalu.txt                           |    0
 .../kannada-pre-processed/1-janapada-kathegalu.csv | 6605 ++++++++++++++++++++
 .../kannada-pre-processed/2-neeti-kathegalu.csv    | 5915 ++++++++++++++++++
 kannada/stop-word

# Task
Evaluate the back-translation accuracy of a placeholder Sarvam AI model from Tamil to English using 10 aligned word pairs (1 from each of the 10 MFD categories) from "anlp-project/mfd2.0.dic" and "anlp-project/tamil/tamil_mfd.dic".

## Load and Preprocess English MFD

### Subtask:
Load the 'anlp-project/mfd2.0.dic' file, filter out comment and category lines (starting with '%' or a digit), and extract the first token as the English word from each remaining line.


In [1]:
import os

file_path = 'anlp-project/mfd2.0.dic'
english_words = []

if os.path.exists(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip() # Remove leading/trailing whitespace
            if line and not (line.startswith('%') or line[0].isdigit()):
                # Extract the first token (word)
                first_token = line.split()[0]
                english_words.append(first_token)
    print(f"Successfully loaded {len(english_words)} English words.")
    print("First 10 extracted words:", english_words[:10])
else:
    print(f"Error: File not found at {file_path}")

Successfully loaded 2104 English words.
First 10 extracted words: ['compassion', 'empathy', 'kindness', 'caring', 'generosity', 'benevolence', 'altruism', 'compassionate', 'nurture', 'gentleness']


## Load and Preprocess Tamil MFD

### Subtask:
Load the 'anlp-project/tamil/tamil_mfd.dic' file, filter out comment and category lines (starting with '%' or a digit), and extract the first token as the Tamil word from each remaining line.

## Load and Preprocess Tamil MFD

### Subtask:
Load the 'anlp-project/tamil/tamil_mfd.dic' file, filter out comment and category lines (starting with '%' or a digit), and extract the first token as the Tamil word from each remaining line.

In [2]:
import os

tamil_file_path = 'anlp-project/tamil/tamil_mfd.dic'
tamil_words = []

if os.path.exists(tamil_file_path):
    with open(tamil_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip() # Remove leading/trailing whitespace
            if line and not (line.startswith('%') or line[0].isdigit()):
                # Extract the first token (word)
                first_token = line.split()[0]
                tamil_words.append(first_token)
    print(f"Successfully loaded {len(tamil_words)} Tamil words.")
    print("First 10 extracted Tamil words:", tamil_words[:10])
else:
    print(f"Error: File not found at {tamil_file_path}")

Successfully loaded 2115 Tamil words.
First 10 extracted Tamil words: ['#', '#', '#', '#', 'கருணை', 'பரிவு', 'கருணை', 'கவனித்தல்', 'தாராள', 'நன்மை']


In [3]:
cleaned_tamil_words = [word for word in tamil_words if word != '#']

print(f"Successfully cleaned {len(tamil_words) - len(cleaned_tamil_words)} non-word entries.")
print(f"Total Tamil words after cleaning: {len(cleaned_tamil_words)}")
print("First 10 cleaned Tamil words:", cleaned_tamil_words[:10])

Successfully cleaned 4 non-word entries.
Total Tamil words after cleaning: 2111
First 10 cleaned Tamil words: ['கருணை', 'பரிவு', 'கருணை', 'கவனித்தல்', 'தாராள', 'நன்மை', 'பிறர்', 'கருணை', 'வளர்த்தல்', 'மென்மை']


In [4]:
import os

# Load English MFD with categories
english_data = []
with open('anlp-project/mfd2.0.dic', 'r', encoding='utf-8') as f:
    for line in f:
        line_orig = line.strip()
        if line_orig and not (line_orig.startswith('%') or line_orig[0].isdigit()):
            # Category is the last token
            parts = line_orig.split()
            if len(parts) >= 2:
                try:
                    category = int(parts[-1])
                    word = ' '.join(parts[:-1])
                    english_data.append((word, category))
                except ValueError:
                    continue

# Load Tamil MFD with categories
tamil_data = []
with open('anlp-project/tamil/tamil_mfd.dic', 'r', encoding='utf-8') as f:
    for line in f:
        line_orig = line.strip()
        if line_orig and not (line_orig.startswith('%') or (line_orig and line_orig[0].isdigit())):
            parts = line_orig.split()
            if len(parts) >= 2:
                try:
                    category = int(parts[-1])
                    word = ' '.join(parts[:-1])
                    if word != '#':
                        tamil_data.append((word, category))
                except ValueError:
                    continue

print(f"English entries: {len(english_data)}")
print(f"Tamil entries: {len(tamil_data)}")

# Category mapping
category_names = {
    1: 'care.virtue',
    2: 'care.vice',
    3: 'fairness.virtue',
    4: 'fairness.vice',
    5: 'loyalty.virtue',
    6: 'loyalty.vice',
    7: 'authority.virtue',
    8: 'authority.vice',
    9: 'sanctity.virtue',
    10: 'sanctity.vice'
}

# Select 1 word from each category
sampled_pairs = []
print("\nSelected words by category:")
print("=" * 60)

for cat_num in sorted(category_names.keys()):
    cat_name = category_names[cat_num]
    # Get first 1 English word from this category
    eng_words = [w for w, c in english_data if c == cat_num][:1]
    # Get first 1 Tamil word from this category
    tam_words = [w for w, c in tamil_data if c == cat_num][:1]

    print(f"\n{cat_name}:")
    for i in range(min(len(eng_words), len(tam_words))):
        eng = eng_words[i]
        tam = tam_words[i]
        sampled_pairs.append((eng, tam))
        print(f"  {eng} -> {tam}")

print(f"\n{'='*60}")
print(f"Total pairs selected: {len(sampled_pairs)}")
print(f"Coverage: {len(category_names)} categories \u00d7 1 word = {len(sampled_pairs)} words")

English entries: 2104
Tamil entries: 2103

Selected words by category:

care.virtue:
  compassion -> கருணை

care.vice:
  harm -> தீங்கு

fairness.virtue:
  equality -> சமத்துவம்

fairness.vice:
  cheat -> ஏமாற்று

loyalty.virtue:
  team player -> குழு விளையாட்டு வீரர்

loyalty.vice:
  traitor -> துரோகி

authority.virtue:
  respect -> மரியாதை

authority.vice:
  disrespect -> அவமரியாதை

sanctity.virtue:
  sanctity -> புனிதத்தன்மை

sanctity.vice:
  impurity -> மாசு

Total pairs selected: 10
Coverage: 10 categories × 1 word = 10 words


In [ ]:
import os
import pickle

# Save as .dic file
dic_file_path = os.path.join('tamil_samples.dic')
with open(dic_file_path, 'w', encoding='utf-8') as f:
    for eng_word, tam_word in sampled_pairs:
        f.write(f"{tam_word}\t{eng_word}\n")
print(f"Saved sampled pairs to {dic_file_path}")

# Save as .pkl file
pkl_file_path = os.path.join('tamil_samples.pkl')
with open(pkl_file_path, 'wb') as f:
    pickle.dump(sampled_pairs, f)
print(f"Saved sampled pairs to {pkl_file_path}")

Saved sampled pairs to tamil_samples.dic
Saved sampled pairs to tamil_samples.pkl


In [13]:
import pickle
import os

pkl_file_path = 'tamil_samples.pkl'

if os.path.exists(pkl_file_path):
    with open(pkl_file_path, 'rb') as f:
        sampled_pairs = pickle.load(f)
    print(f"Successfully loaded {len(sampled_pairs)} sampled pairs from {pkl_file_path}")
    print("First 5 loaded pairs:")
    for i, (eng, tam) in enumerate(sampled_pairs[:5]):
        print(f"{i+1}. English: {eng}, Tamil: {tam}")
else:
    print(f"Error: Pickle file not found at {pkl_file_path}")


Successfully loaded 10 sampled pairs from tamil_samples.pkl
First 5 loaded pairs:
1. English: compassion, Tamil: கருணை
2. English: harm, Tamil: தீங்கு
3. English: equality, Tamil: சமத்துவம்
4. English: cheat, Tamil: ஏமாற்று
5. English: team player, Tamil: குழு விளையாட்டு வீரர்


**Reasoning**:
To install the `transformers` library, I will use the `pip install` command within a code cell.



In [23]:
pip install transformers

**Reasoning**:
Now that the `transformers` library is installed, I will define a function to perform Tamil to English translation using the specified Sarvam AI model ('sarvamai/sarvam-translate') by loading `AutoTokenizer` and `AutoModelForCausalLM`.



In [24]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Model details
model_name = "sarvamai/sarvam-translate"
tgt_lang = "English"  # English in Latin script
source_lang = "Tamil"  # Tamil in Tamil script

# Check for GPU availability
device = 0 if torch.cuda.is_available() else -1
print(f"Using device: {'cuda' if device == 0 else 'cpu'}")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


import requests

def sarvam_ai_tamil_to_english(tamil_word):
    """
    Uses Sarvam's official API for concise translations
    """
    url = "https://api.sarvam.ai/translate"

    payload = {
        "input": tamil_word,
        "source_language_code": "ta-IN",  # Tamil
        "target_language_code": "en-IN",  # English
        "speaker_gender": "Male",
        "mode": "formal",
        "model": "mayura:v1"
    }

    headers = {
        "Content-Type": "application/json",
        "API-Subscription-Key": "sk_19fh2qvp_sAvUR5GoQReBtxKqwrKW1P5V"
    }

    response = requests.post(url, json=payload, headers=headers)
    result = response.json()

    return result.get("translated_text", "").strip()


print("Sarvam AI translation model and function loaded.")


Using device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Sarvam AI translation model and function loaded.


In [25]:
back_translated_tamil = []
for english_word, tamil_word in sampled_pairs:
  back_translated_english = sarvam_ai_tamil_to_english(tamil_word)
  back_translated_tamil.append({
        "original_english": english_word,
        "tamil_word": tamil_word,
        "translated_english": back_translated_english
    })
print(back_translated_tamil)


[{'original_english': 'compassion', 'tamil_word': 'கருணை', 'translated_english': 'Karunai'}, {'original_english': 'harm', 'tamil_word': 'தீங்கு', 'translated_english': 'Harmful'}, {'original_english': 'equality', 'tamil_word': 'சமத்துவம்', 'translated_english': 'Equality'}, {'original_english': 'cheat', 'tamil_word': 'ஏமாற்று', 'translated_english': 'Deceit'}, {'original_english': 'team player', 'tamil_word': 'குழு விளையாட்டு வீரர்', 'translated_english': 'Team player'}, {'original_english': 'traitor', 'tamil_word': 'துரோகி', 'translated_english': 'Traitor'}, {'original_english': 'respect', 'tamil_word': 'மரியாதை', 'translated_english': 'Respect'}, {'original_english': 'disrespect', 'tamil_word': 'அவமரியாதை', 'translated_english': 'Disrespect'}, {'original_english': 'sanctity', 'tamil_word': 'புனிதத்தன்மை', 'translated_english': 'Purity'}, {'original_english': 'impurity', 'tamil_word': 'மாசு', 'translated_english': 'Masu'}]


## Back-Translation

In [26]:
print(f"{'Original English':<20} | {'Tamil Word':<20} | {'Back-Translated English':<30}")
print(f"{'':-<20}-+-{'':-<20}-+-{'':-<30}")
for item in back_translated_tamil:
    print(f"{item['original_english']:<20} | {item['tamil_word']:<20} | {item['translated_english']:<30}")

Original English     | Tamil Word           | Back-Translated English       
---------------------+----------------------+-------------------------------
compassion           | கருணை                | Karunai                       
harm                 | தீங்கு               | Harmful                       
equality             | சமத்துவம்            | Equality                      
cheat                | ஏமாற்று              | Deceit                        
team player          | குழு விளையாட்டு வீரர் | Team player                   
traitor              | துரோகி               | Traitor                       
respect              | மரியாதை              | Respect                       
disrespect           | அவமரியாதை            | Disrespect                    
sanctity             | புனிதத்தன்மை         | Purity                        
impurity             | மாசு                 | Masu                          




## Back-Translation Results

| Original English | Tamil Word | Back-Translated English | Semantic Match
|:-----------------|:-------------|:-------------------|:--------------|
| compassion | கருணை | Karunai  | No |
| harm   | தீங்கு | Harmful    | Yes |
| equality  | சமத்துவம்  | Equality   | Yes|
| cheat  | ஏமாற்று  | Deceit   | Yes |
| team player   | குழு விளையாட்டு வீரர்  | Team player  | Yes|
| traitor     | துரோகி  | Traitor  | Yes|
| respect        | மரியாதை     | Respect  | Yes|
| disrespect         | அவமரியாதை| Disrespect   | Yes |
| sanctity         | புனிதத்தன்மை  |Purity  | Yes|
| impurity         | மாசு| Masu        | No |


In [27]:
back_translated_tamil = []
for english_word, tamil_word in sampled_pairs:
  back_translated_english = sarvam_ai_tamil_to_english(tamil_word)
  back_translated_tamil.append({
        "original_english": english_word,
        "tamil_word": tamil_word,
        "translated_english": back_translated_english
    })
print(back_translated_tamil)


[{'original_english': 'compassion', 'tamil_word': 'கருணை', 'translated_english': 'Karunai'}, {'original_english': 'harm', 'tamil_word': 'தீங்கு', 'translated_english': 'Harmful'}, {'original_english': 'equality', 'tamil_word': 'சமத்துவம்', 'translated_english': 'Equality'}, {'original_english': 'cheat', 'tamil_word': 'ஏமாற்று', 'translated_english': 'Deceit'}, {'original_english': 'team player', 'tamil_word': 'குழு விளையாட்டு வீரர்', 'translated_english': 'Team player'}, {'original_english': 'traitor', 'tamil_word': 'துரோகி', 'translated_english': 'Traitor'}, {'original_english': 'respect', 'tamil_word': 'மரியாதை', 'translated_english': 'Respect'}, {'original_english': 'disrespect', 'tamil_word': 'அவமரியாதை', 'translated_english': 'Disrespect'}, {'original_english': 'sanctity', 'tamil_word': 'புனிதத்தன்மை', 'translated_english': 'Purity'}, {'original_english': 'impurity', 'tamil_word': 'மாசு', 'translated_english': 'Masu'}]


## Machine-Human Translation Comparison

| English      | Sarvam AI        | Human 1            | Human 2            | Agreement Type | MT Performance | Category | Notes |
| ------------ | ---------------- | ------------------ | ------------------ | -------------- | -------------- | -------- | ----- |
| compassion   | கருணை           | -                  | -                  | -              | -              | care.virtue | - |
| harm         | தீங்கு          | -                  | -            | -              | -              | care.vice | - |
| equality     | சமத்துவம்        | -                  | -               | -              | -              | fairness.virtue | - |
| cheat        | ஏமாற்று         | -                  | -                | -              | -              | fairness.vice | - |
| team player  | குழு விளையாட்டு வீரர் | -          | -                  | -              | -              | loyalty.virtue | - |
| traitor      | துரோகி          | -                  | -                  | -              | -              | loyalty.vice | - |
| respect      | மரியாதை         | -                  | -                  | -              | -              | authority.virtue | - |
| disrespect   | அவமரியாதை       | -                  | -                  | -              | -              | authority.vice | - |
| sanctity     | புனிதத்தன்மை     | -                  | -                  | -              | -              | sanctity.virtue | - |
| impurity     | மாசு            | -                  | -                  | -              | -              | sanctity.vice | - |

## Summary & Analysis

A systematic evaluation of 10 Tamil translations from the English Moral Foundation Dictionary (MFD) was conducted using Sarvam AI machine translation, validated through back-translation and dual human annotation. The evaluation includes 1 word from each of the 10 MFD categories, ensuring balanced coverage across all moral foundations.



In [28]:
%pip install nltk

In [30]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Download necessary NLTK data if not already present
try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

english_stopwords = set(stopwords.words('english'))
stemmer = PorterStemmer()

def normalize_text(text):
    """
    Converts string to lowercase, strips whitespace, removes basic punctuation, removes stopwords, and applies stemming.
    """
    text = text.lower() # Convert to lowercase
    text = text.strip() # Remove leading/trailing whitespace
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize, remove stopwords, and stem
    words = text.split()
    filtered_words = [word for word in words if word not in english_stopwords]
    stemmed_words = [stemmer.stem(word) for word in filtered_words]
    return ' '.join(stemmed_words)

exact_matches = 0
total_words = len(back_translated_tamil)

for word in back_translated_tamil:
    normalized_original_english = normalize_text(word['original_english'])
    normalized_translated_english = normalize_text(word['translated_english'])

    if normalized_original_english == normalized_translated_english:
        exact_matches += 1

accuracy_score = exact_matches / total_words

print(f"Total sampled pairs: {total_words}")
print(f"Exact matches: {exact_matches}")
print(f"Exact-match accuracy: {accuracy_score:.2f}")

Total sampled pairs: 10
Exact matches: 6
Exact-match accuracy: 0.60
